Kaggle Project with MLBox

### Step 1. Load and preprocess datasets

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from mlbox.preprocessing import *
from mlbox.optimisation import *
from mlbox.prediction import *

import warnings
warnings.simplefilter('ignore')

C:\Users\y.li\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\y.li\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\y.li\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\y.li\AppData\Roaming\Python\Python36\site-pack

In [2]:
trans_train_df = pd.read_csv(r'C:\Users\y.li\OneDrive\Study\kaggle_IEEE_fraud\ieee-fraud-detection\train_transaction.csv')
trans_test_df = pd.read_csv(r'C:\Users\y.li\OneDrive\Study\kaggle_IEEE_fraud\ieee-fraud-detection\test_transaction.csv')

KeyboardInterrupt: 

In [17]:
identity_train_df = pd.read_csv(r'C:\Users\y.li\OneDrive\Study\kaggle_IEEE_fraud\ieee-fraud-detection\train_identity.csv')
identity_test_df = pd.read_csv(r'C:\Users\y.li\OneDrive\Study\kaggle_IEEE_fraud\ieee-fraud-detection\test_identity.csv')

FileNotFoundError: [Errno 2] File b'...\\ieee-fraud-detection\\train_identity.csv' does not exist: b'...\\ieee-fraud-detection\\train_identity.csv'

In [ ]:
trans_train_df.shape, identity_train_df.shape, trans_test_df.shape, identity_test_df.shape

In [ ]:
train_df = pd.merge(trans_train_df, identity_train_df, on = 'TransactionID', how = 'left')

In [ ]:
test_df = pd.merge(trans_test_df, identity_test_df, on = 'TransactionID', how = 'left')

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df.to_csv(r'C:\Users\y.li\OneDrive\Study\kaggle_IEEE_fraud\ieee-fraud-detection\train.csv', index = False)
test_df.to_csv(r'C:\Users\y.li\OneDrive\Study\kaggle_IEEE_fraud\ieee-fraud-detection\test.csv', index = False)            

---------------------

In [5]:
train_df = pd.read_csv(r'C:\Users\y.li\OneDrive\Study\kaggle_IEEE_fraud\ieee-fraud-detection\train.csv')
test_df = pd.read_csv(r'C:\Users\y.li\OneDrive\Study\kaggle_IEEE_fraud\ieee-fraud-detection\test.csv')

In [6]:
train_df.columns

Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=434)

In [7]:
train_df.shape, test_df.shape

((590540, 434), (506691, 433))

### 2. Add dataset path and label

In [8]:
paths = [r'C:\Users\y.li\OneDrive\Study\kaggle_IEEE_fraud\ieee-fraud-detection\train.csv', r'C:\Users\y.li\OneDrive\Study\kaggle_IEEE_fraud\ieee-fraud-detection\test.csv']
target_name = "isFraud"

### 3. Read and clean the train and test

In [9]:
rd = Reader(sep = ",")
df = rd.train_test_split(paths, target_name)   


reading csv : C:\Users\y.li\OneDrive\Study\kaggle_IEEE_fraud\ieee-fraud-detection\train.csv ...
cleaning data ...
CPU time: 124.55311369895935 seconds

reading csv : C:\Users\y.li\OneDrive\Study\kaggle_IEEE_fraud\ieee-fraud-detection\test.csv ...
cleaning data ...
CPU time: 111.50930738449097 seconds

> Number of common features : 433

gathering and crunching for train and test datasets ...
reindexing for train and test datasets ...
dropping training duplicates ...
dropping constant variables on training set ...

> Number of categorical features: 31
> Number of numerical features: 402
> Number of training samples : 590540
> Number of test samples : 506691

> Top sparse features (% missing values on train set):
id_24    99.2
id_25    99.1
id_07    99.1
id_08    99.1
id_21    99.1
dtype: float64

> Task : classification
0.0    569877
1.0     20663
Name: isFraud, dtype: int64

encoding target ...


### 4. Remove drift variables
* Drift variables: can be used to predict train and test
* Algorithm: Univaritate drift estimation and greedy recursive drift elimination

In [10]:
dft = Drift_thresholder()
df = dft.fit_transform(df) 


computing drifts ...
CPU time: 2723.4491283893585 seconds

> Top 10 drifts

('TransactionDT', 1.0)
('TransactionID', 1.0)
('id_31', 0.3446333923946341)
('id_13', 0.226036499359215)
('D15', 0.18413864080899267)
('D11', 0.16496057285844623)
('D10', 0.1495683066573068)
('D4', 0.14546431770297752)
('V78', 0.13896968704820534)
('V45', 0.13837172101464446)

> Deleted variables : ['TransactionDT', 'TransactionID']
> Drift coefficients dumped into directory : save


### 5. Set hyperparameter space

In [11]:
space = {'ne__numerical_strategy':{"search":"choice",
                                   "space":['mean','median']},
         
         'ne__categorical_strategy':{"search":"choice",
                                     "space":['<NULL>']},
         
         'ce__strategy':{"search":"choice",
                         "space":['label_encoding','entity_embedding','random_projection']},
         
         'fs__strategy':{"search":"choice",
                         "space":['l1','rf_feature_importance','variance']},
         
        'est__strategy':{"search":"choice",
                                  "space":["LightGBM"]},  
         
        'est__n_estimators':{"search":"choice",
                                  "space":[150, 250,500,1000,3000]},
         
        'est__colsample_bytree':{"search":"uniform",
                                  "space":[0.8,0.95]},
         
        'est__subsample':{"search":"uniform",
                                  "space":[0.8,0.95]},
         
        'est__max_depth':{"search":"choice",
                                  "space":[3,5,7,9]},
         
        'est__learning_rate':{"search":"uniform",
                                  "space":[0.03, 0.3]} 
    
        }



### 6. Optimize the best hyper parameter

In [12]:
#from sklearn.metrics import roc_auc_score
opt = Optimiser(scoring = 'roc_auc', n_folds = 5)
best_params = opt.optimise(space, df, 50)

##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 'median', 'categorical_strategy': '<NULL>'}
>>> CA ENCODER :{'strategy': 'random_projection'}   
>>> FEATURE SELECTOR :{'strategy': 'rf_feature_importance', 'threshold': 0.3}
>>> ESTIMATOR :{'strategy': 'LightGBM', 'colsample_bytree': 0.8496657698403252, 'learning_rate': 0.2812251629545792, 'max_depth': 9, 'n_estimators': 250, 'subsample': 0.9079561554251527, 'boosting_type': 'gbdt', 'class_weight': None, 'importance_type': 'split', 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': True, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'nthread': -1, 'seed': 0}
MEAN SCORE : roc_auc = -inf                         
VARIANCE : nan (fold 1 = -inf, fold 2 = -inf, fold 3 = -in

W0917 17:22:00.336917 21328 logger.py:77] [MemorizedFunc(func=<function _fit_transform_one at 0x0000025F29B2D378>, location=save\joblib)]: Exception while loading results for _fit_transform_one(<mlbox.encoding.na_encoder.NA_encoder object at 0x0000025FC81DEB70>,          C1  C10  C11  C12   C13  C14   C2   C3   C4   C5  ...     id_29  \
0       1.0  0.0  2.0  0.0   1.0  1.0  1.0  0.0  0.0  0.0  ...       NaN   
1       1.0  0.0  1.0  0.0   1.0  1.0  1.0  0.0  0.0  0.0  ...       NaN   
2       1.0  0.0  1.0  0.0   1.0  1.0  1.0  0.0  0.0  0.0  ...       NaN   
3       2.0  0.0  1.0  0.0  25.0  1.0  5.0  0.0  0.0  0.0  ...       NaN   
4       1.0  1.0  1.0  0.0   1.0  1.0  1.0  0.0  0.0  0.0  ...  NotFound   
...     ...  ...  ...  ...   ...  ...  ...  ...  ...  ...  ...       ...   
590534  1.0  0.0  1.0  1.0   1.0  1.0  1.0  0.0  1.0  0.0  ...  NotFound   
590535  2.0  0.0  1.0  0.0   3.0  2.0  1.0  0.0  0.0  1.0  ...       NaN   
590536  1.0  0.0..., 
0         0
1         0
2      

MEAN SCORE : roc_auc = -inf                                      
VARIANCE : nan (fold 1 = -inf, fold 2 = -inf, fold 3 = -inf, fold 4 = -inf, fold 5 = -inf)
CPU time: 89.6402428150177 seconds                               
##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 'median', 'categorical_strategy': '<NULL>'}
>>> CA ENCODER :{'strategy': 'label_encoding'}                   
>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.3}       
>>> ESTIMATOR :{'strategy': 'LightGBM', 'colsample_bytree': 0.8768218231584046, 'learning_rate': 0.05966333119236211, 'max_depth': 7, 'n_estimators': 250, 'subsample': 0.9285005222210327, 'boosting_type': 'gbdt', 'class_weight': None, 'importance_type': 'split', 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.

W0917 17:23:20.873050 21328 logger.py:77] [MemorizedFunc(func=<function _fit_transform_one at 0x0000025F29B2D378>, location=save\joblib)]: Exception while loading results for _fit_transform_one(<mlbox.encoding.na_encoder.NA_encoder object at 0x0000025FC81837B8>,          C1  C10  C11  C12   C13  C14   C2   C3   C4   C5  ...     id_29  \
0       1.0  0.0  2.0  0.0   1.0  1.0  1.0  0.0  0.0  0.0  ...       NaN   
1       1.0  0.0  1.0  0.0   1.0  1.0  1.0  0.0  0.0  0.0  ...       NaN   
2       1.0  0.0  1.0  0.0   1.0  1.0  1.0  0.0  0.0  0.0  ...       NaN   
3       2.0  0.0  1.0  0.0  25.0  1.0  5.0  0.0  0.0  0.0  ...       NaN   
4       1.0  1.0  1.0  0.0   1.0  1.0  1.0  0.0  0.0  0.0  ...  NotFound   
...     ...  ...  ...  ...   ...  ...  ...  ...  ...  ...  ...       ...   
590534  1.0  0.0  1.0  1.0   1.0  1.0  1.0  0.0  1.0  0.0  ...  NotFound   
590535  2.0  0.0  1.0  0.0   3.0  2.0  1.0  0.0  0.0  1.0  ...       NaN   
590536  1.0  0.0..., 
0         0
1         0
2      

MEAN SCORE : roc_auc = -inf                                      
VARIANCE : nan (fold 1 = -inf, fold 2 = -inf, fold 3 = -inf, fold 4 = -inf, fold 5 = -inf)
CPU time: 80.31087732315063 seconds                              
##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 'median', 'categorical_strategy': '<NULL>'}
>>> CA ENCODER :{'strategy': 'label_encoding'}                   
>>> FEATURE SELECTOR :{'strategy': 'variance', 'threshold': 0.3} 
>>> ESTIMATOR :{'strategy': 'LightGBM', 'colsample_bytree': 0.8102658866932242, 'learning_rate': 0.15649601321048773, 'max_depth': 5, 'n_estimators': 1000, 'subsample': 0.8479622337939038, 'boosting_type': 'gbdt', 'class_weight': None, 'importance_type': 'split', 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0

...

MEAN SCORE : roc_auc = -inf                                                       
VARIANCE : nan (fold 1 = -inf, fold 2 = -inf, fold 3 = -inf, fold 4 = -inf, fold 5 = -inf)
CPU time: 788.4507050514221 seconds                                               
##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 'mean', 'categorical_strategy': '<NULL>'}  
>>> CA ENCODER :{'strategy': 'random_projection'}                                 
>>> FEATURE SELECTOR :{'strategy': 'rf_feature_importance', 'threshold': 0.3}     
>>> ESTIMATOR :{'strategy': 'LightGBM', 'colsample_bytree': 0.8415289131995417, 'learning_rate': 0.11729545876627813, 'max_depth': 5, 'n_estimators': 1000, 'subsample': 0.9258324250243756, 'boosting_type': 'gbdt', 'class_weight': None, 'importance_type': 'split', 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_jobs': -1, 'num_

W0917 20:24:40.234923 21328 logger.py:77] [MemorizedFunc(func=<function _fit_transform_one at 0x0000025F29B2D378>, location=save\joblib)]: Exception while loading results for _fit_transform_one(<mlbox.encoding.categorical_encoder.Categorical_encoder object at 0x0000025FDE8CEFD0>, 
         C1  C10  C11  C12   C13  C14   C2   C3   C4   C5  ...     id_29  \
0       1.0  0.0  2.0  0.0   1.0  1.0  1.0  0.0  0.0  0.0  ...    <NULL>   
1       1.0  0.0  1.0  0.0   1.0  1.0  1.0  0.0  0.0  0.0  ...    <NULL>   
2       1.0  0.0  1.0  0.0   1.0  1.0  1.0  0.0  0.0  0.0  ...    <NULL>   
3       2.0  0.0  1.0  0.0  25.0  1.0  5.0  0.0  0.0  0.0  ...    <NULL>   
4       1.0  1.0  1.0  0.0   1.0  1.0  1.0  0.0  0.0  0.0  ...  NotFound   
...     ...  ...  ...  ...   ...  ...  ...  ...  ...  ...  ...       ...   
590534  1.0  0.0  1.0  1.0   1.0  1.0  1.0  0.0  1.0  0.0  ...  NotFound   
590535  2.0  0.0  1.0  0.0   3.0  2.0  1.0  0.0  0.0  1.0  ...    <NULL>   
590536  1.0  0.0..., 
0         0


MEAN SCORE : roc_auc = -inf                                                       
VARIANCE : nan (fold 1 = -inf, fold 2 = -inf, fold 3 = -inf, fold 4 = -inf, fold 5 = -inf)
CPU time: 159.23828601837158 seconds                                              
##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 'median', 'categorical_strategy': '<NULL>'}
>>> CA ENCODER :{'strategy': 'label_encoding'}                                    
>>> FEATURE SELECTOR :{'strategy': 'variance', 'threshold': 0.3}                  
>>> ESTIMATOR :{'strategy': 'LightGBM', 'colsample_bytree': 0.8713771528638804, 'learning_rate': 0.08234356993083211, 'max_depth': 3, 'n_estimators': 250, 'subsample': 0.840974501903653, 'boosting_type': 'gbdt', 'class_weight': None, 'importance_type': 'split', 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_jobs': -1, 'num_le

W0917 20:27:20.800657 21328 logger.py:77] [MemorizedFunc(func=<function _fit_transform_one at 0x0000025F29B2D378>, location=save\joblib)]: Exception while loading results for _fit_transform_one(<mlbox.encoding.categorical_encoder.Categorical_encoder object at 0x0000025FDF2B92B0>, 
         C1  C10  C11  C12   C13  C14   C2   C3   C4   C5  ...     id_29  \
0       1.0  0.0  2.0  0.0   1.0  1.0  1.0  0.0  0.0  0.0  ...    <NULL>   
1       1.0  0.0  1.0  0.0   1.0  1.0  1.0  0.0  0.0  0.0  ...    <NULL>   
2       1.0  0.0  1.0  0.0   1.0  1.0  1.0  0.0  0.0  0.0  ...    <NULL>   
3       2.0  0.0  1.0  0.0  25.0  1.0  5.0  0.0  0.0  0.0  ...    <NULL>   
4       1.0  1.0  1.0  0.0   1.0  1.0  1.0  0.0  0.0  0.0  ...  NotFound   
...     ...  ...  ...  ...   ...  ...  ...  ...  ...  ...  ...       ...   
590534  1.0  0.0  1.0  1.0   1.0  1.0  1.0  0.0  1.0  0.0  ...  NotFound   
590535  2.0  0.0  1.0  0.0   3.0  2.0  1.0  0.0  0.0  1.0  ...    <NULL>   
590536  1.0  0.0..., 
0         0


MEAN SCORE : roc_auc = -inf                                                       
VARIANCE : nan (fold 1 = -inf, fold 2 = -inf, fold 3 = -inf, fold 4 = -inf, fold 5 = -inf)
CPU time: 153.57669472694397 seconds                                              
##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 'median', 'categorical_strategy': '<NULL>'}
>>> CA ENCODER :{'strategy': 'random_projection'}                                 
>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.3}                        
>>> ESTIMATOR :{'strategy': 'LightGBM', 'colsample_bytree': 0.8775417237955011, 'learning_rate': 0.1923339943207109, 'max_depth': 7, 'n_estimators': 150, 'subsample': 0.8256700918860206, 'boosting_type': 'gbdt', 'class_weight': None, 'importance_type': 'split', 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_jobs': -1, 'num_le

### 7. Predict test dataset

In [ ]:
predict = Predictor()
predict.fit_predict(best_params,df)


fitting the pipeline ...
CPU time: 72.7819766998291 seconds
